In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

np.random.seed(42)

n_customers = 1000
n_orders = 8000

customer_ids = np.random.randint(1000, 2000, n_orders)

start_date = datetime(2023, 1, 1)
purchase_dates = [start_date + timedelta(days=np.random.randint(0, 365)) for _ in range(n_orders)]

order_values = np.random.randint(200, 5000, n_orders)

data = pd.DataFrame({
    "CustomerID": customer_ids,
    "InvoiceDate": purchase_dates,
    "OrderValue": order_values
})

data.to_csv("ecommerce_rfm_dummy.csv", index=False)

print("Dummy dataset created.")


Dummy dataset created.


In [5]:
import pandas as pd
import datetime as dt

df = pd.read_csv("ecommerce_rfm_dummy.csv")
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

today = dt.datetime(2024, 1, 1)

rfm = df.groupby("CustomerID").agg({
    "InvoiceDate": lambda x: (today - x.max()).days,
    "CustomerID": "count",
    "OrderValue": "sum"
})

rfm.columns = ["Recency", "Frequency", "Monetary"]

rfm["R_score"] = pd.qcut(rfm["Recency"], 4, labels=[4,3,2,1])
rfm["F_score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 4, labels=[1,2,3,4])
rfm["M_score"] = pd.qcut(rfm["Monetary"], 4, labels=[1,2,3,4])

rfm["RFM_Score"] = (
    rfm["R_score"].astype(int) +
    rfm["F_score"].astype(int) +
    rfm["M_score"].astype(int)
)

def segment(score):
    if score >= 10:
        return "High Value"
    elif score >= 7:
        return "Potential Loyalist"
    elif score >= 4:
        return "At Risk"
    else:
        return "Lost"

rfm["Segment"] = rfm["RFM_Score"].apply(segment)

rfm.head()


,Recency,Frequency,Monetary,R_score,F_score,M_score,RFM_Score,Segment
CustomerID,,,,,,,,
1000,1,11,18713,4,4,2,10,High Value
1001,4,7,20685,4,2,3,9,Potential Loyalist
1002,8,8,14594,4,2,2,8,Potential Loyalist
1003,2,5,16186,4,1,2,7,Potential Loyalist
1004,49,9,15553,2,3,2,7,Potential Loyalist
